# Выбор локации для скважины

Задача: Добывающей компании нужно решить, где бурить новую скважину. Разработать модель для анализа и выбора наиболее подходящей скважины, где добыча принесёт наибольшую прибыль. 

Анализ будем проводить техникой *Bootstrap.*

Информация о шагах для выбора локации:
- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных** 

Данные геологоразведки трёх регионов находятся в файлах: 
- /datasets/geo_data_0.csv 
- /datasets/geo_data_1.csv
- /datasets/geo_data_2.csv

Содержание файлов: 
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек;
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
try:
    data1 = pd.read_csv('/datasets/geo_data_0.csv')
    print('Файл 1 успешно загружен!')
    data2 = pd.read_csv('/datasets/geo_data_1.csv')
    print('Файл 2 успешно загружен!')
    data3 = pd.read_csv('/datasets/geo_data_2.csv')
    print('Файл 3 успешно загружен!')
except:
    print('Ошибка загрузки...')


Файл 1 успешно загружен!
Файл 2 успешно загружен!
Файл 3 успешно загружен!


Посмотрим основную информацию по каждому региону.

Регион №1

In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


Регион №2

In [6]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
data2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


Регион №3

In [9]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
data3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [11]:
data3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Данные полные, пустых значений нет. В каждом регионе данные о 100 тыс. скважен. В данных только количественные признаки, которые достаточно одинаково распределены, следовательно нам не нужно кодирование, но на всякий случай сделаем масштабирование.

## Обучение и проверка модели

### Разобьем данные на обучающую и валидационную выборки в соотношении 75:25

Разобьем данные на признаки и целевые признаки. Id скважен для обучения нам не нужны, так что их удалим.

In [12]:
target1 = data1['product']
features1 = data1.drop(['id','product'], axis=1)

target2 = data2['product']
features2 = data2.drop(['id','product'], axis=1)

target3 = data3['product']
features3 = data3.drop(['id','product'], axis=1)

Разобьем данные на выборки.

In [13]:
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size = 0.25, random_state=12345) 
print('Размеры выборок 1:')
print('Тренировочная 1',features_train1.shape)
print('Валидационная 1',features_valid1.shape)

features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size = 0.25, random_state=12345) 
print('Размеры выборок 2:')
print('Тренировочная 2',features_train2.shape)
print('Валидационная 2',features_valid2.shape)

features_train3, features_valid3, target_train3, target_valid3 = train_test_split(
    features3, target3, test_size = 0.25, random_state=12345) 
print('Размеры выборок 3:')
print('Тренировочная 3',features_train3.shape)
print('Валидационная 3',features_valid3.shape)

Размеры выборок 1:
Тренировочная 1 (75000, 3)
Валидационная 1 (25000, 3)
Размеры выборок 2:
Тренировочная 2 (75000, 3)
Валидационная 2 (25000, 3)
Размеры выборок 3:
Тренировочная 3 (75000, 3)
Валидационная 3 (25000, 3)


Масштабируем признаки.

In [14]:
features_train1

,f0,f1,f2
27212,0.022450,0.951034,2.197333
7866,1.766731,0.007835,6.436602
62041,0.724514,0.666063,1.840177
70185,-1.104181,0.255268,2.026156
82230,-0.635263,0.747990,6.643327
...,...,...,...
4094,1.863680,-0.298123,1.621324
85412,-1.162682,-0.014822,6.819941
2177,0.862688,-0.403776,1.867662
77285,0.846235,-0.489533,1.058786


In [15]:
pd.options.mode.chained_assignment = None
numeric = ['f0','f1','f2']

scaler = StandardScaler()
scaler.fit(features1[numeric])
features_train1[numeric] = scaler.transform(features_train1[numeric])
features_valid1[numeric] = scaler.transform(features_valid1[numeric])

scaler.fit(features2[numeric])
features_train2[numeric] = scaler.transform(features_train2[numeric])
features_valid2[numeric] = scaler.transform(features_valid2[numeric])

scaler.fit(features3)
features_train3[numeric] = scaler.transform(features_train3[numeric])
features_valid3[numeric] = scaler.transform(features_valid3[numeric])

### Обучим модель и сделаем предсказания на валидационной выборке

По условию задачи мы используем только линейную регрессию.

In [16]:
model1 = LinearRegression()
model1.fit(features_train1, target_train1)

model2 = LinearRegression()
model2.fit(features_train2, target_train2)

model3 = LinearRegression()
model3.fit(features_train3, target_train3)

LinearRegression()

### Сохраним предсказания на валидационной выборке

In [17]:
predicted_valid1 = pd.Series(data=model1.predict(features_valid1))
predicted_valid2 = pd.Series(data=model2.predict(features_valid2))
predicted_valid3 = pd.Series(data=model3.predict(features_valid3))

### Средний запас предсказанного сырья и RMSE модели

In [18]:
print('Cредний запас в первом регионе:', predicted_valid1.mean())
print('RMSE model1:', mean_squared_error(target_valid1, predicted_valid1)**0.5)
print()

print('Cредний запас в первом регионе:', predicted_valid2.mean())
print('RMSE model2:', mean_squared_error(target_valid2, predicted_valid2)**0.5)
print()

print('Cредний запас в первом регионе:', predicted_valid3.mean())
print('RMSE model3:', mean_squared_error(target_valid3, predicted_valid3)**0.5)
print()

Cредний запас в первом регионе: 92.59256778438035
RMSE model1: 37.5794217150813

Cредний запас в первом регионе: 68.72854689544602
RMSE model2: 0.893099286775617

Cредний запас в первом регионе: 94.96504596800489
RMSE model3: 40.02970873393434



Проверим модели на адекватность. Сравним работу наших моделей с моделями, предсказывающими среднее.


In [19]:
predictions_mean1 = pd.Series(target1.mean(), index=target1.index)
print('RMSE model1:', mean_squared_error(target1, predictions_mean1)**0.5)

predictions_mean2 = pd.Series(target2.mean(), index=target2.index)
print('RMSE model2:', mean_squared_error(target2, predictions_mean2)**0.5)

predictions_mean3 = pd.Series(target3.mean(), index=target3.index)
print('RMSE model3:', mean_squared_error(target3, predictions_mean3)**0.5)

RMSE model1: 44.2884696928441
RMSE model2: 45.94419317228633
RMSE model3: 44.74969731878746


### Вывод

Мы подготовили данные, разделили их на целевой и обучающие признаки, обучили три модели для каждого региона, RMSE у них оказалась меньше, чем у константной модели, проверку на адекватность они прошли. 

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраним в отдельных переменных.

In [24]:
UNIT_INCOME = 450_000 #доход за единицу продукта
REGION_BUDGET = 10_000_000_000 #бюджет для разработки в одном регионе
BEST_COUNT = 200 #количество лучших точек, которое нам нужно

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. 

In [25]:
enough_product = REGION_BUDGET/BEST_COUNT/UNIT_INCOME
print('Достаточный объем продукта для безубыточной точки равен',enough_product, 'тыс. баррелей')

Достаточный объем продукта для безубыточной точки равен 111.11111111111111 тыс. баррелей


Средние запасы даже в лучших регионах не дотягивают до 100, но мы ищем лучшие точки.

### Вывод

Мы сохранили все ключевые значение и подготовились к расчету прибыли. 

## Расчёт прибыли и рисков 

### Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [22]:
def revenue(predicted, target):
    target = target.reset_index(drop=True)
    predicted = predicted.sort_values(ascending=False)[:BEST_COUNT]
    rez = UNIT_INCOME*sum(target.loc[predicted.index]) - REGION_BUDGET
    return rez

### Посчитаем риски и прибыль для каждого региона

In [23]:
values1 = []
values2 = []
values3 = []
state = np.random.RandomState(12345)
for i in range(1000):
    subsample1 = predicted_valid1.sample(n = 500, replace=True, random_state=state)
    values1.append(revenue(subsample1, target_valid1))
    
    subsample2 = predicted_valid2.sample(n = 500, replace=True, random_state=state)
    values2.append(revenue(subsample2, target_valid2))
    
    subsample3 = predicted_valid3.sample(n = 500, replace=True, random_state=state)
    values3.append(revenue(subsample3, target_valid3))

values1 = pd.Series(values1)
lower1 = values1.quantile(0.025) 
upper1 = values1.quantile(0.975) 
print('Средняя прибыль 200 лучших точек первого региона:', values1.mean())
print('95% интервал:',lower1, upper1)
print('Риски:', 100*(values1<0).mean(), '%') # считаем долю отрицательных среди 200 лучших
print()

values2 = pd.Series(values2)
lower2 = values2.quantile(0.025) 
upper2 = values2.quantile(0.975) 
print('Средняя прибыль 200 лучших точек второго региона:', values2.mean())
print('95% интервал:',lower2, upper2)
print('Риски:', 100*(values2<0).mean(), '%')
print()

values3 = pd.Series(values3)
lower3 = values3.quantile(0.025) 
upper3 = values3.quantile(0.975) 
print('Средняя прибыль 200 лучших точек третьего региона:', values3.mean())
print('95% интервал:',lower3, upper3)
print('Риски:', 100*(values3<0).mean(), '%')
print()

Средняя прибыль 200 лучших точек первого региона: 384260328.7812565
95% интервал: -148694265.84742263 909153837.0354805
Риски: 8.3 %

Средняя прибыль 200 лучших точек второго региона: 455016071.7470537
95% интервал: 48072046.16942068 848709853.4682904
Риски: 1.5 %

Средняя прибыль 200 лучших точек третьего региона: 380893248.2818159
95% интервал: -146453456.04119882 945355944.8165586
Риски: 9.0 %



### Вывод

По нашим подсчетам вышло, что наиболее безопасным по убыткам и прибыльным регионом является Регион №2. Его средняя прибыль 455 млн, а 95% значений прибыли попадают в интервал 48 тыс. - 848 млн. рублей. Риск убытка составляет 1.5%. 

Для улучшения проекта (для избежания ошибок и если регионов будет больше) следуюет написать отдельные фукции для избегания повторов кусков кода. 